<h1>Floating Centroid Method with Fuzzy C Means Algorithm</h1>

In [1]:
import numpy as np

In [2]:
def activate(z,activation) :
    

In [3]:
def forward_propagation_step(a_prev, weights, biases, activation) :
    '''
    params :
        a_prev : (C[L-1],m)
        weights : (C[L],C[L-1])
        biases : (C[L],1)
    '''
    z = np.dot(weights, a_prev) + biases
    a = activate(z, activation) 
    
    return z,a

In [4]:
def forward_propagation(activations, z_values, weights, biases, a_funcs) :
    '''
    params :
        activations :  a list of activations of all layers
        z_values : a list of z values of all layers (dummy for 0th layer)
        weights : list of weights of all layers
        biases : list of biases of all layers
        a_funcs : list of activation functions for all layers (dummy for 0th layer)
    '''
    L = len(activations)
    
    for i in range(1, L) :
        weight = weights[i]
        bias = biases[i]
        activation = activations[i - 1]
        a_func = a_funcs[i]
        
        z, a = forward_propagation_step(activation, weight, bias, a_func)
        z_values[i] = z
        activations[i] = a

In [5]:
def d_activate(z, activation) :
    pass

In [6]:
def backward_propagation_step(l, learning_rate, z_value, weights, biases, 
                              activation, delta, a_next, lamb,C_S, C_N, a, epsilon) :
    '''
    params :
        l : layer index
        learning rate : learning rate
        z_value : (C[L], m)
        weights : (C[L],C[L-1])
        biases : (C[L-1],1)
        activation : activation function
        delta : (C[L+1],m)
    '''
    L = None # TODO : update
    delta = None
    
    if l == L :
        delta_l = d_activate(z_value,activation) * ((C_S - a) - (epsilon * (C_N - a)))
        return delta, None, None
    else :
        delta_l = np.dot(weights[l + 1],delta) * d_activate(z_value,activation) 
        
    dw = learning_rate * (delta_l * a_next - (lamb * weight))
    weights = weights - dw 
    
    db = -learning_rate * delta_l
    biases = biases - db
    
    return delta_l, weights, bisaes

In [8]:
def backward_propagation(activations, z_values, weights, biases, learning_rate, epsilon, lamb, C_S, C_N) :
    
    L = len(weights)
    delta = None
    
    i = L
    while i > 0 :
        
        weight = weights[i]
        bias = biases[i]
        
        a_next = None
        if i < L :
            a_next = activations[i+1]
        
        z_value = z_values[i]
        a_func = a_funcs[i]
        
        delta, w , b = backward_propagation_step(i, learning_rate, z_value, weight, bias, activation, 
                                                 delta, a_next, lamb, C_S, C_N, activations[i], epsilon)
        
        weights[i] = w
        biases[i] = b
        
        i -= 1

In [ ]:
def dist(X, V) :
    '''
    parmas :
        X : (m,N)
        V : (C,N)
    '''
    N = X.shape[1]
    m = X.shape[0]
    C = V.shape[0]
    distancce = np.zeros((m, N, C))
    d = np.zeros((m,C))
    
    for i in range(0, C) :
        distance[:, :, i] = X - V[i,:]
        distance[:, : ,i] = np.pow(distance[:,:,i],2)
        d[:,i] = np.sum(distance[:,:,i], axis = 1)
    
    return d

In [ ]:
def calculate_membership(X , V) :
    
    m = X.shape[0]
    C = V.shape[0]
    
    d = dist(X,V)
    u = np.zeros((m,C))
    u = np.pow(np.pow(d, 1/ (r - 1)) / np.pow(np.sum(d,axis = 1), 1 / (r - 1)), -1)
    return u

In [ ]:
def move_centroids(X,u) :
    
    V = np.zeros((C,N))
    denom = np.sum(np.pow(u,r), axis = 0)
    V = np.dot(X.transpose(),np.pow(u,r)).transpose()
    V = V / denom
    
    return V

In [ ]:
def build_clusters(X_input, r, max_iters, num_clusters, Y, num_classes) :
    '''
    returns :
        cluster_count : (C,num_classes)
                        # of classes per cluster
    '''
    X = X_input.transpose()
    m = X.shape[0]
    N = X.shape[1]
    C = num_clusters
    
    V = np.zeros((C,N))
    
    for i in range(max_iters) :
        u = calculate_membership(X, V)
        V = move_centroids(X,u)    
    
    clusters = np.argmax(u, axis = 1) 
    
    cluster_count = np.zeros((C,num_classes))
    
    for i in range(0,m) :
        cluster_count[clusters[i]][Y[i]] += 1
        
    return V, c, clusters, cluster_count

In [ ]:
def color_centroids(clusters, cluster_count) :
    
    C = cluster_count.shape[0]
    color = [0 for i in range(0, C)]
    
    for i in range(0, C) :
        color[i] = np.argmax(cluster_count, axis = 1)
        
    return color

In [ ]:
def euclidean_distance(a , b) :
    
    c = a - b
    c = np.power(c, 2)
    s = np.sqrt(np.sum(c, axis = 1))
    
    return s

In [ ]:
def get_Cs(color, cluster_count, centroids, mapped_val, Y) :
    '''
    params : 
            Y : actual class of i'th example
    '''
    
    C = cluster_count.shape[0]
    
    C_S = 0
    C_N = 0
    min_S = None
    ind_S = None
    min_N = None
    ind_N = None
    
    for i in range(0, C) :
        d = euclidean_distance(mapped_val, centroids[i])
        if Y == color[i] :
            if min_S == None :
                min_S = d
                ind_S = i
            elif min_S > d :
                min_S = d
                ind_S = i
        else :
            if min_N == None :
                min_N = d
                ind_N = i
            elif min_N > d :
                min_N = d
                ind_N = i
    
    if min_S == None :
        
        max_percent = 0
        for i in range(0, C) :
            d = euclidean_distance(mapped_val, centroids[i])
            percent = cluster_count[i][Y] / np.sum(cluster_count[i], axis = 1)
            
            if percent > max_percent :
                max_percent = percent
                min_S = d
                ind_S = i
            
    if min_N == None :
        
        max_percent = 0
        for i in range(0, C) :
            d = euclidean_distance(mapped_val, centroids[i])
            total = np.sum(cluster_count[i], axis = 1)
            percent = (total - cluster_count[i][Y]) / total
            
            if percent > max_percent :
                max_percent = percent
                min_N = d
                ind_N = i
    
    return centroids[ind_S], centroids[ind_N]

In [ ]:
def gradient_descent(X, Y, max_iters) :
    
    for i in range(0, max_iters) :
        
        forward_propagation()
        backward_propagation()
        build_clusters()
        color_centroids()